In [1]:
import sys
sys.path.append(r"C:\Users\CristianRojas\Desktop\Challenge\Challenge Data Engineer\Seccion_1\app")

In [1]:
from app.database import SessionLocal, init_db
from app.models import Department, Job, HiredEmployee

In [2]:
!pip install -r requirements.txt


In [ ]:
# pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [3]:
# uvicorn app.main:app --reload

In [4]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "API funcionando correctamente"}

read_root()

{'message': 'API funcionando correctamente'}

In [5]:
from app.database import init_db
init_db()

In [6]:
from sqlalchemy import inspect
from app.database import engine

inspector = inspect(engine)
print(inspector.get_table_names())

['departments', 'hired_employees', 'jobs']


In [1]:
import sys
sys.path.append("C:/Users/CristianRojas/Desktop/Challenge/Challenge Data Engineer/Seccion_1/")
from app.database import SessionLocal
from app.models import Department, Job, HiredEmployee

# Ejemplo: iniciar sesión y consultar datos
db = SessionLocal()
print(db.query(Department).all())

[<app.models.Department object at 0x000001B0A3EAB760>, <app.models.Department object at 0x000001B0A3EAB700>, <app.models.Department object at 0x000001B0A3EAB820>, <app.models.Department object at 0x000001B0A3EAB880>, <app.models.Department object at 0x000001B0A3EAB8E0>, <app.models.Department object at 0x000001B0A3EAB940>, <app.models.Department object at 0x000001B0A3EABA90>, <app.models.Department object at 0x000001B0A3EABAF0>, <app.models.Department object at 0x000001B0A3EABB50>, <app.models.Department object at 0x000001B0A3EABD30>, <app.models.Department object at 0x000001B0A3EABD90>, <app.models.Department object at 0x000001B0A3EABDF0>]


In [2]:
departments = db.query(Department).all()
for dept in departments:
    print({"id": dept.id, "department": dept.department})


{'id': 1, 'department': 'Product Management'}
{'id': 2, 'department': 'Sales'}
{'id': 3, 'department': 'Research and Development'}
{'id': 4, 'department': 'Business Development'}
{'id': 5, 'department': 'Engineering'}
{'id': 6, 'department': 'Human Resources'}
{'id': 7, 'department': 'Services'}
{'id': 8, 'department': 'Support'}
{'id': 9, 'department': 'Marketing'}
{'id': 10, 'department': 'Training'}
{'id': 11, 'department': 'Legal'}
{'id': 12, 'department': 'Accounting'}


In [3]:
import pandas as pd
from app.database import engine    # necesitamos el engine

In [4]:
# Ejemplo: leemos la tabla completa de jobs
jobs = pd.read_sql_query("SELECT * FROM jobs", engine)
display(jobs)

,id,job
0,1,Marketing Assistant
1,2,VP Sales
2,3,Biostatistician IV
3,4,Account Representative II
4,5,VP Marketing
...,...,...
178,179,Software Engineer II
179,180,Statistician IV
180,181,Programmer Analyst I
181,182,Account Representative I


In [5]:

# Ejemplo: leemos la tabla completa de hired_employees
hired_employees = pd.read_sql_query("SELECT * FROM hired_employees", engine)
display(hired_employees)

,id,name,datetime,department_id,job_id
0,1,Harold Vogt,2021-11-07 02:48:42.000000,2,96
1,3,Lyman Hadye,2021-09-01 23:27:38.000000,5,52
2,4,Lotti Crowthe,2021-10-01 13:04:21.000000,12,71
3,5,Gretna Lording,2021-10-10 22:22:17.000000,6,80
4,6,Marlow Antecki,2021-04-23 23:45:42.000000,6,95
...,...,...,...,...,...
1938,1995,Goran Yong,2021-08-22 21:57:53.000000,2,83
1939,1996,Cirstoforo Youings,2021-04-01 17:48:42.000000,3,23
1940,1997,Wilek Yurkevich,2021-05-04 12:19:50.000000,2,132
1941,1998,Jerry Yven,2021-10-03 14:12:50.000000,7,100


In [6]:
# Ejemplo: leemos la tabla completa de departamentos
df_departments = pd.read_sql_query("SELECT * FROM departments", engine)
display(df_departments)

,id,department
0,1,Product Management
1,2,Sales
2,3,Research and Development
3,4,Business Development
4,5,Engineering
5,6,Human Resources
6,7,Services
7,8,Support
8,9,Marketing
9,10,Training


In [8]:


# 1.2. Métrica: hires por trimestre (SQL puro adaptado a SQLite)
sql = """
    SELECT
      d.department,
      j.job,
      SUM(CASE WHEN CAST(strftime('%m', he.datetime) AS INTEGER) BETWEEN 1 AND 3 THEN 1 ELSE 0 END) AS Q1,
      SUM(CASE WHEN CAST(strftime('%m', he.datetime) AS INTEGER) BETWEEN 4 AND 6 THEN 1 ELSE 0 END) AS Q2,
      SUM(CASE WHEN CAST(strftime('%m', he.datetime) AS INTEGER) BETWEEN 7 AND 9 THEN 1 ELSE 0 END) AS Q3,
      SUM(CASE WHEN CAST(strftime('%m', he.datetime) AS INTEGER) BETWEEN 10 AND 12 THEN 1 ELSE 0 END) AS Q4
    FROM hired_employees he
    JOIN departments d ON he.department_id = d.id
    JOIN jobs j        ON he.job_id        = j.id
    WHERE strftime('%Y', he.datetime) = '2021'
    GROUP BY d.department, j.job
    ORDER BY d.department, j.job
    """
df_quarters = pd.read_sql_query(sql, engine)
display(df_quarters)

,department,job,Q1,Q2,Q3,Q4
0,Accounting,Account Representative IV,1,0,0,0
1,Accounting,Actuary,0,1,0,0
2,Accounting,Analyst Programmer,0,0,1,0
3,Accounting,Budget/Accounting Analyst III,0,1,0,0
4,Accounting,Cost Accountant,0,1,0,0
...,...,...,...,...,...,...
928,Training,Teacher,0,2,0,0
929,Training,Technical Writer,0,0,1,0
930,Training,VP Product Management,1,0,0,0
931,Training,VP Quality Control,0,1,0,0


In [9]:
sql2 = """
WITH hires_per_dept AS (
  SELECT
    d.id,
    d.department,
    COUNT(*) AS hired
  FROM hired_employees he
  JOIN departments d ON he.department_id = d.id
  WHERE strftime('%Y', he.datetime) = '2021'
  GROUP BY d.id, d.department
),
avg_hires AS (
  SELECT AVG(hired) AS mean_hires
  FROM hires_per_dept
)
SELECT
  h.id,
  h.department,
  h.hired
FROM hires_per_dept h
CROSS JOIN avg_hires a
WHERE h.hired > a.mean_hires
ORDER BY h.hired DESC;
"""

# Ejecutamos la consulta y cargamos el resultado en un DataFrame
df_above_mean = pd.read_sql_query(sql2, engine)

# Mostramos el resultado
display(df_above_mean)

,id,department,hired
0,8,Support,216
1,5,Engineering,205
2,6,Human Resources,201
3,7,Services,200
4,4,Business Development,185
5,3,Research and Development,148
6,9,Marketing,142


In [10]:
db.close()